In [ ]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
import pandas as pd

### Obteniedo metadatos cursos

In [ ]:
columns = ['title', 'description', 'difficulty_level', 'duration', 'keywords', 'url']
df = pd.DataFrame(columns = columns)

In [ ]:
def get_metadata_page(url): 
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    h1_tags = soup('h1')
    title = h1_tags[0].get_text()

    description = soup.find('div', {'class': 'content-inner'}).get_text()

    h4_tags = soup('h4')
    difficulty_level = ''
    duration = ''
    language = h4_tags[-1].get_text()
    for h4 in h4_tags:    
        if (re.search("Level", str(h4))):
            difficulty_level = h4.get_text()
        if (re.search("Approx", str(h4))):
            duration = h4.get_text()

    try:
        entradas = soup.find('div', {'class': 'Box_120drhm-o_O-displayflex_poyjc-o_O-wrap_rmgg7w'})
        keywords = ''
        for span in entradas:
            keywords += span.get_text() + ', '
    except:
        keywords = ''

    diccionario = {'title': title, 'description': description, 'difficulty_level': difficulty_level,
                   'duration': duration, 'keywords': keywords, 'url': url}
    return diccionario

### Iterar sobre todos los cursos

In [ ]:
for i in range(1, 101):
    try:
        url = 'http://www.coursera.org/courses?query=&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D=' + str(i) + '&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        lis = soup.findAll('li', {'class': 'ais-InfiniteHits-item'})
        j = 1
        for li in lis:
            links = li.findAll('a', attrs={'href': re.compile("/learn/")})
            if(len(links) > 0):
                url = 'https://www.coursera.org' + links[0].get('href')
                print(i, 'li'+str(j), url)
                j += 1
                diccionario = get_metadata_page(url)
                df = df.append(diccionario, ignore_index=True)
    except:
        pass

In [ ]:
df

In [ ]:
df.to_json("metadataCoursera2.json", orient='records')
df.to_excel("metadataCoursera2.xlsx", "data")